<h1>Finger print classifier Notebook</h1>
	

<h2>First let us import Our necessary packages</h2>

In [ ]:
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,Dropout
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import pickle
from keras.initializers import glorot_uniform
import keras.backend as K
import matplotlib.pyplot as plt


<h3>Our Preprocessing steps are:</h3>

<h4>1-Read text files beside every image and create .csv file which contain picture info Related with it's id</h4>

In [ ]:
def createMapDataset(path):
    onlyfiles = ([f.replace('.txt', '') for f in listdir(path) if isfile(join(path, f))])

    dataframe['id'] = onlyfiles
    content = []
    gender = []
    cl = []
    for i in range(len(onlyfiles)):
        with open(path + '/' + str(onlyfiles[i]) + '.txt') as f:
            content.append(f.readlines())
    for c in content:
        c = [x.strip() for x in c]
        gender.append(c[0].split(' ')[1])
        cl.append(c[1].split(' ')[1])
    dataframe['gender'] = gender
    dataframe['class'] = cl
    dataframe.to_csv('dataframe.csv')

<h4>2-We Want to split Our Images into Train , CrossValidation and Test sets
        But We Need To ensure Data shuffled for Good Representation of all classes in train, test and validation sets</h4>

In [ ]:
def splitImages(path):
   
    df = pd.read_csv(path)
    ids = pd.DataFrame(df)
    ids_shuffeled = ids.sample(frac=1, random_state=1)
    ids_shuffeled['gender'] = ids_shuffeled['gender'].map({'M': 0, 'F': 1})
    ids_shuffeled['class'] = ids_shuffeled['class'].map({'A': 0, 'L': 1, 'T':2, 'R':3, 'W':4})
    train_df_x = pd.DataFrame(data=ids_shuffeled[:2285].drop('class', 1))
    crossVal_df_x = pd.DataFrame(data=ids_shuffeled[2285:2885].drop('class', 1))
    test_df_x = pd.DataFrame(data=ids_shuffeled[2885:].drop('class', 1))
    train_labels = pd.DataFrame(data = ids_shuffeled['class'][:2285])
    crossVal_labels = pd.DataFrame(data=ids_shuffeled['class'][2285:2885])
    test_labels = pd.DataFrame(data=ids_shuffeled['class'][2885:])
    train_df_x.to_csv('Train.csv', index=False)
    crossVal_df_x.to_csv('val.csv')
    test_df_x.to_csv('test.csv')

    return train_df_x, crossVal_df_x, test_df_x, train_labels, crossVal_labels,test_labels

<h4>3-Create Subdirectories for each class (from our  a,w,r,l,t classes) because that is the format which our image datagenerator class waiting for</h4>


In [ ]:
def split_classes(path):
    df = pd.read_csv(path, index_col=False)
    class_a=[] ; class_t=[]; class_r=[]; class_w=[]; class_l = []
    class_a.extend(df.id[(df['class'] == 'A')])
    class_t.extend(df.id[(df['class'] == 'T')])
    class_r.extend(df.id[(df['class'] == 'R')])
    class_w.extend(df.id[(df['class'] == 'W')])
    class_l.extend(df.id[(df['class'] == 'L')])

    return class_a, class_t, class_r, class_w, class_l

In [ ]:
def create_subfolders(a,t,r,w,l):
    class_a = a
    class_t = t
    class_r = r
    class_w = w
    class_l = l
    for i in class_a:
        srcfile = 'Dataset/sd04/png_txt/Untitled Folder/' + str(i) + '.png'
        dstroot = 'Dataset/sd04/png_txt/figs/test/a'
        assert not os.path.isabs(srcfile)
        dstdir = os.path.join(dstroot, os.path.dirname(str(i) + '.png'))

        shutil.copy(srcfile, dstdir)
    for j in class_t:
        srcfile = 'Dataset/sd04/png_txt/Untitled Folder/' + str(j) + '.png'
        dstroot = 'Dataset/sd04/png_txt/figs/test/t'
        assert not os.path.isabs(srcfile)
        dstdir = os.path.join(dstroot, os.path.dirname(str(j) + '.png'))

        shutil.copy(srcfile, dstdir)
    for k in class_r:
        srcfile = 'Dataset/sd04/png_txt/Untitled Folder/' + str(k) + '.png'
        dstroot = 'Dataset/sd04/png_txt/figs/test/r'
        assert not os.path.isabs(srcfile)
        dstdir = os.path.join(dstroot, os.path.dirname(str(k) + '.png'))

        shutil.copy(srcfile, dstdir)

    for l in class_l:
        srcfile = 'Dataset/sd04/png_txt/Untitled Folder/' + str(l) + '.png'
        dstroot = 'Dataset/sd04/png_txt/figs/test/l'
        assert not os.path.isabs(srcfile)
        dstdir = os.path.join(dstroot, os.path.dirname(str(l) + '.png'))

        shutil.copy(srcfile, dstdir)

    for w in class_w:
        srcfile = 'Dataset/sd04/png_txt/Untitled Folder/' + str(w) + '.png'
        dstroot = 'Dataset/sd04/png_txt/figs/test/w'
        assert not os.path.isabs(srcfile)
        dstdir = os.path.join(dstroot, os.path.dirname(str(w) + '.png'))

        shutil.copy(srcfile, dstdir)

<h4>4-last step using imageDatageneartor class from keras and apply some data augmentation on images to build powerful convnet model with little amount of data</h4>


In [ ]:
K.set_image_data_format('channels_last')
K.set_learning_phase(1)
train_generator = ImageDataGenerator(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'

        ).flow_from_directory(
        directory='Dataset/sd04/png_txt/figs/train',  # this is the target directory
        target_size=(64, 64),  # all images will be resized to 64x64
        batch_size=16,
        class_mode='categorical',
        color_mode='rgb',)

crossVal_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        directory='Dataset/sd04/png_txt/figs/test',  
        target_size=(64, 64),  
        batch_size=16,
        class_mode='categorical',
        color_mode='rgb')

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        directory='Dataset/sd04/png_txt/figs/validation',  
        target_size=(64, 64),  
        batch_size=16,
        class_mode='categorical',
        color_mode='rgb')

<h2>What are Convolutional Neural Networks and why are they important?</h2>



<h4>Convolutional Neural Networks (ConvNets or CNNs) are a category of Neural Networks that have proven very effective in areas such as image recognition and classification. ConvNets have been successful in identifying faces, objects and traffic signs apart from powering vision in robots and self driving cars.</h4>

<img src="images/model.png"/>

<h2>We will discuss the following topics:</h2>

<h3>1-ConvLayer

2-Pool Layer</h3>

<h2>1-Convolutional Layer:
A convolution layer transforms an input volume into an output volume of different size, as shown below.</h2>


<img src="images/conv_nn.png" />

<h2>2-PoolingLayer:
The pooling (POOL) layer reduces the height and width of the input. It helps reduce computation, as well as helps make feature detectors more invariant to its position in the input. The two types of pooling layers are:
Max-pooling layer: slides an ( f,ff,f ) window over the input and stores the max value of the window in the output.
Average-pooling layer: slides an ( f,ff,f ) window over the input and stores the average value of the window in the output.</h2>

<img src ="images/max_pool.png" />

<img src="images/ave-pool.png" / >

<h1>Why Resnet50?</h1>

<h3>The main benefit of a very deep network is that it can represent very complex functions. It can also learn features at many different levels of abstraction, from edges (at the lower layers) to very complex features (at the deeper layers). However, using a deeper network doesn't always help. A huge barrier to training them is vanishing gradients: very deep networks often have a gradient signal that goes to zero quickly, thus making gradient descent unbearably slow. More specifically, during gradient descent, as you backprop from the final layer back to the first layer, you are multiplying by the weight matrix on each step, and thus the gradient can decrease exponentially quickly to zero (or, in rare cases, grow exponentially quickly and "explode" to take very large values).</h3>

<img src ="images/graph.png" />

<h1>What's new in Resnet50</h1>

<img src ="images/main_pth.png" />

<img src ="images/shortcut_path.png" />

<h3>The image on the left shows the "main path" through the network. The image on the right adds a shortcut to the main path. By stacking these ResNet blocks on top of each other, you can form a very deep network.
shortcut also makes it very easy for one of the blocks to learn an identity function. This means that you can stack on additional ResNet blocks with little risk of harming training set performance. (There is also some evidence that the ease of learning an identity function--even more than skip connections helping with vanishing gradients--accounts for ResNets' remarkable performance.)</h3>

<h2>Blocks in our Resnet50:</h2>

<h3>1-The identity block</h3>

<img src="images/iden.png" />

<h3>2-Conv Block</h3>

<img src="images/conv.png" />

<h1>The fullcode of Resnet50 to classify fingerprints </h1>

In [ ]:
def identity_block(X, f, filters, stage, block):


    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)


    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,  name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,  name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)


    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):


    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(F1, (1, 1), strides=(s, s), name=conv_name_base + '2a',
               kernel_initializer=glorot_uniform(seed=0), padding='valid')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)


    X = Conv2D(F2, (f, f), strides=(1, 1), name=conv_name_base + '2b',
               kernel_initializer=glorot_uniform(seed=0), padding='same')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(F3, (1, 1), strides=(1, 1), name=conv_name_base + '2c',
               kernel_initializer=glorot_uniform(seed=0), padding='valid')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, strides=(s, s), kernel_size=(1, 1),
                        padding='valid',name=conv_name_base + '1',
                        kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)


    return X

In [ ]:
def ResNet50(input_shape=(64, 64, 3), classes=5):
  
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X,f=3,block='a',filters = [128, 128, 256],stage=3,s=2)
    X = identity_block(X, f = 3, filters = [128, 128, 256], stage = 3, block = 'b')
    X = identity_block(X, f = 3, filters = [128, 128, 256], stage = 3, block = 'c')
    X = identity_block(X, f = 3, filters = [128, 128, 256], stage = 3, block = 'd')

    X = convolutional_block(X,f=3,block='a',filters = [256, 256, 1024],stage=4,s=2)
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'b')
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'c')
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'd')
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'e')
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'f')

    X = convolutional_block(X, f=3, block='a', filters=[512, 512, 2048],stage=5,s=2)
    X = identity_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block = 'b')
    X = identity_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block = 'c')

    X = AveragePooling2D(pool_size=(2, 2), name="avg_pool")(X)


    X = Flatten()(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes),
              kernel_initializer=glorot_uniform(seed=0))(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [ ]:
model = ResNet50(input_shape = (64, 64, 3), classes = 5)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
Hist = model.fit_generator(train_generator, steps_per_epoch=2285, validation_data=crossVal_generator,
                    validation_steps=1115,epochs=150)

<h1>Here we plot some statistics about loss,accuracy and num of epochs</h1>

In [ ]:
my_dict_final = {}  # Create an empty dictionary
with open('History10.pickle', 'rb') as f:
    my_dict_final.update(pickle.load(f))   # Update contents of file1 to the dictionary

with open('History9.pickle', 'rb') as f:
    my_dict_final.update(pickle.load(f)) # Update contents of file2 to the dictionary
with open('History8.pickle', 'rb') as f:
    my_dict_final.update(pickle.load(f))
with open('History7.pickle', 'rb') as f:
    my_dict_final.update(pickle.load(f))
with open('History6.pickle', 'rb') as f:
    my_dict_final.update(pickle.load(f))
with open('History5.pickle', 'rb') as f:
    my_dict_final.update(pickle.load(f))
with open('History4.pickle', 'rb') as f:
    my_dict_final.update(pickle.load(f))
with open('History3.pickle', 'rb') as f:
    my_dict_final.update(pickle.load(f))
with open('History2.pickle', 'rb') as f:
    my_dict_final.update(pickle.load(f))
with open('History.pickle', 'rb') as f:
    my_dict_final.update(pickle.load(f))


his = pickle.load(open('History10.pickle', 'rb'))


# summarize history for accuracy
plt.plot(my_dict_final['acc'])
plt.plot(my_dict_final['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(my_dict_final['loss'])
plt.plot(my_dict_final['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

<img src="images/Figure_1.png" />

<img src="images/Figure_2.png" />